In [1]:
%load_ext autoreload
%matplotlib widget
%autoreload 2
import sys
sys.path.append("../")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sumo


BANZUKE_RANGE = list(range(200001, 201801))
# BANZUKE_RANGE = None
db = sumo.connect()

# Sumo Prediction Model

## Preprocessing

First, load the data from the database and preprocess into a sensible table of data in pandas.

In [2]:
banzukes = sumo.load_banzuke(db, BANZUKE_RANGE)
history_df = sumo.load_history(db, BANZUKE_RANGE)
history_df = history_df.join(banzukes, on=['bid', 'rid'])

rikishi = sumo.load_rikishi(db)
rikishi.head()

,,hoshi,prizes,rank,score,weight,height
bid,rid,,,,,,
198803,1,[],[],Mz,"[0, 0]",152,200
198805,1,"[shiro, yasumi, yasumi, shiro, yasumi, shiro, ...",[],Jk19e,"[6, 1]",152,200
198807,1,"[shiro, yasumi, yasumi, shiro, kuro, yasumi, y...",[],Jd97e,"[5, 2]",152,200
198809,1,"[yasumi, shiro, yasumi, shiro, yasumi, kuro, s...",[],Jd52e,"[5, 2]",154.5,202
198811,1,"[shiro, yasumi, shiro, yasumi, shiro, yasumi, ...",[],Jd15w,"[6, 1]",154.5,202


## Create Feature Matrix

Create a feature matrix with the following features for each prediction. All properties should be calculated excluding the "current" basho:
 - Total number of previous wins
 - Total number of previous losses
 - Average number of absent days
 - Average number of losses.
 - Average number of wins.
 - Average number of absent days
 - Sliding window average wins
 - Sliding window average losses
 - Sliding window average absent days
 - Current ranking

In [3]:
from sumo import elo

def create_feature_matrix(df):

    def career_total(x, column):
        return x.groupby(['rid'])[column].apply(lambda x: x.shift().fillna(0).cumsum())
    
    def cum_basho_total(x, column):
        return x.groupby(['bid', 'rid'])[column].apply(lambda x: x.shift().fillna(0).cumsum())
    
    def last_value(x, column):
        return x.groupby('rid')[column].apply(lambda x: x.shift().fillna(0))
    
    return (df.sort_values(['bid', 'day', 'rid'])
               .reset_index()
               .assign(career_wins=lambda x: career_total(x, 'result'),
                       current_basho_wins=lambda x: cum_basho_total(x, 'result'),
                       last_result=lambda x: last_value(x, 'result'),
                       num_rank=lambda x: pd.Categorical(x['rank'].values, categories=sumo.SUMO_RANKS, ordered=True).codes,
                      )
                .set_index(['bid', 'day', 'rid'])
           )
features = create_feature_matrix(history_df)

features = (features.join(rikishi, on=['bid', 'rid'], rsuffix='_r')
             .assign(weight=lambda x: x.weight.astype(float).fillna(value=x.weight.astype(float).mean()),
                     height=lambda x: x.height.astype(float).fillna(value=x.height.astype(float).mean()))
            )

features = features.join(features, on=['bid', 'day', 'opponent'], rsuffix='_opp', how='right')
features.index = features.index.set_names(['bid', 'day', 'rid'])

def get_mask(x):
    x.reset_index(inplace=True)
    bout_rikishi = x[['rid', 'opponent']]
    sorted_values = np.sort(bout_rikishi.values, axis=1)
    return ~pd.DataFrame(sorted_values, index=x.index).duplicated()

# remove duplicate pairs like:
# > rikishi1 - rikishi2
# > rikishi2 - rikishi1
features = (features
                .groupby(['bid', 'day'])
                .apply(lambda x: x.loc[get_mask(x)])
                .drop(['day', 'bid'], axis=1)
                .reset_index()
                .set_index(['bid', 'day', 'rid'])
                .drop(['level_2'], axis=1)
             )
features = features.dropna()

features = elo.rankings(features, 'rid', 'opponent', 'result')
features = features.assign(elo_diff=lambda x: x.elo - x.elo_opp,
                           rank_diff=lambda x: x.num_rank - x.num_rank_opp,
                           weight_diff=lambda x: x.weight - x.weight_opp,
                           height_diff=lambda x: x.height - x.height_opp,
                           BMI=lambda x: x.weight / ((x.height/100.) **2.),
                           BMI_opp=lambda x: x.weight_opp / ((x.height_opp/100.) **2.),
                           BMI_diff=lambda x: x.BMI - x.BMI_opp,
                           wins_last_basho_diff=lambda x: x.wins_last_basho - x.wins_last_basho_opp,
                           absent_last_basho_diff=lambda x: x.absent_last_basho - x.absent_last_basho_opp,
                           result=lambda x: x.result.astype(float)
                          )
features

kimarite  opponent  result rank      shikona  wins  loss  \
bid    day rid                                                               
200001 1   1          40.0        29     1.0    Y      Akebono  11.0   4.0   
           2          74.0        38     1.0    Y   Takanohana  12.0   3.0   
           4          74.0         7     1.0    Y  Musashimaru   2.0   2.0   
           5          40.0        12     1.0    S   Takanonami  10.0   5.0   
           6          74.0        34     1.0    S     Musoyama  13.0   2.0   
           8          33.0        14     1.0    O  Chiyotaikai   9.0   6.0   
           9          29.0        25     1.0   M4   Akinoshima   7.0   8.0   
           10          9.0        26     1.0   M4  Tochinonada   6.0   9.0   
           11          8.0        33     0.0   M6     Aogiyama   4.0  10.0   
           13         74.0        31     1.0    K    Tosanoumi   8.0   7.0   
           16         40.0        18     1.0   M9   Minatofuji   8.0   7.0   
           17          8.0        87     1.0   M8   Tamakasuga   8.0   7.0   
           19          9.0       842     0.0    S   Tochiazuma   8.0   7.0   
           20         74.0        35     1.0   M5   Shikishima   7.0   8.0   
           36         40.0        40     1.0   M8     Asanosho   7.0   8.0   
           41         75.0        54     1.0  M13   Kyokutenho  11.0   4.0   
           43         74.0        58     1.0  M14   Kinkaiyama   6.0   9.0   
           47         41.0        63     1.0  M11  Chiyotenzan   9.0   6.0   
           50          8.0        71     0.0  M11   Tokitsuumi   9.0   6.0   
       2   1          40.0        38     1.0    Y      Akebono  11.0   4.0   
           2          69.0        19     1.0    Y   Takanohana  12.0   3.0   
           4          74.0        29     0.0    Y  Musashimaru   2.0   2.0   
           5          40.0        34     1.0    S   Takanonami  10.0   5.0   
           6          41.0        13     1.0    S     Musoyama  13.0   2.0   
           7          56.0        12     1.0    S         Kaio   7.0   8.0   
           8          74.0       842     0.0    O  Chiyotaikai   9.0   6.0   
           9          56.0        26     0.0   M4   Akinoshima   7.0   8.0   
           10         40.0        25     1.0   M4  Tochinonada   6.0   9.0   
           11          8.0        20     1.0   M6     Aogiyama   4.0  10.0   
           14         75.0        31     1.0   M2   Kotonowaka   6.0   9.0   
...                    ...       ...     ...  ...          ...   ...   ...   
201711 14  2832       40.0     12210     0.0   M5     Arawashi   8.0   7.0   
           5944        8.0      6463     1.0   M1    Tamawashi  11.0   4.0   
           5967       40.0     12107     0.0    S    Yoshikaze   6.0   9.0   
           6468       74.0     11728     1.0    O       Goeido   9.0   6.0   
           6473       40.0      6642     0.0   M2    Tochiozan   4.0  11.0   
           6491       56.0      6753     1.0  M10        Ikioi   9.0   6.0   
           6596       40.0     11786     1.0  M15    Nishikigi   7.0   8.0   
           6599       74.0     11845     1.0   M6  Tochinoshin   9.0   6.0   
           9079       41.0     12291     1.0  M14     Kotoyuki   8.0   7.0   
           11784       5.0     12130     0.0  M15     Myogiryu   6.0   8.0   
           11785      50.0     12191     0.0   M6   Chiyoshoma   7.0   8.0   
           11934      56.0     12144     1.0   M2  Chiyotairyu   7.0   8.0   
           12043      54.0     12239     1.0    K       Onosho   8.0   7.0   
       15  89         67.0     11785     1.0  M13   Aminishiki   8.0   7.0   
           1123       69.0      6468     1.0    Y       Hakuho  14.0   1.0   
           1226       74.0     12107     1.0    K  Kotoshogiku   6.0   9.0   
           1284       54.0      6642     0.0  M13     Takekaze   7.0   8.0   
           2832       74.0      6491     0.0   M5     Arawashi   8.0   7.0   
           5944       40.0     12239    

# Examining Features

In [4]:
plt.close("all")
fig, ax = plt.subplots(figsize=(15,7))
features.boxplot(by='rank', column=['elo'], ax=ax)

FigureCanvasNbAgg()

In [22]:
fig, ax = plt.subplots(figsize=(15,7))
features.plot(by='day', kind='hist', y='rank_diff', ax=ax, bins=30)

FigureCanvasNbAgg()

# Model Training & Prediction

In [167]:
drop_columns = ['shikona', 'rank', 'wins', 'loss', 'absent', 'result', 'opponent', 'score', 'rank_r', 'hoshi', 'prizes']
drop_columns += [name + '_opp' for name in drop_columns]
# mat = mat.dropna()
mat = features.drop(drop_columns, axis=1)

from sklearn import preprocessing

X = mat.values
min_max_scaler = preprocessing.MinMaxScaler()
X = min_max_scaler.fit_transform(X)

y = features.result.values.astype(int)
mat.head()

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


kimarite  wins_last_basho  absent_last_basho  career_wins  \
bid    day rid                                                               
192701 1   3652     103.0              0.0                0.0          0.0   
           3663     106.0              0.0                0.0          0.0   
           3666      77.0              0.0                0.0          0.0   
           3668      93.0              0.0                0.0          0.0   
           3671      85.0              0.0                0.0          0.0   

                 current_basho_wins  last_result  num_rank weight height  \
bid    day rid                                                             
192701 1   3652                 0.0          0.0       0.0      0      0   
           3663                 0.0          0.0      18.0      0      0   
           3666                 0.0          0.0       0.0      0      0   
           3668                 0.0          0.0       6.0      0      0   
           3671                 0.0          0.0       7.0      0      0   

                 kimarite_opp           ...            last_result_opp  \
bid    day rid                          ...                              
192701 1   3652           103           ...                          0   
           3663           106           ...                          0   
           3666            77           ...                          0   
           3668            93           ...                          0   
           3671            85           ...                          0   

                 num_rank_opp  weight_opp  height_opp     elo  elo_opp  \
bid    day rid                                                           
192701 1   3652             9           0           0  1000.0   1000.0   
           3663            15           0           0  1000.0   1000.0   
           3666            11           0           0  1000.0   1000.0   
           3668            12           0           0  1000.0   1000.0   
           3671             3           0           0  1000.0   1000.0   

                elo_diff rank_diff  wins_last_basho_diff  \
bid    day rid                                             
192701 1   3652      0.0      -9.0                   0.0   
           3663      0.0       3.0                   0.0   
           3666      0.0     -11.0                   0.0   
           3668      0.0      -6.0                   0.0   
           3671      0.0       4.0                   0.0   

                 absent_last_basho_diff  
bid    day rid                           
192701 1   3652                     0.0  
           3663                     0.0  
           3666                     0.0  
           3668                     0.0  
           3671                     0.0  

[5 rows x 24 columns]

In [164]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

def baseline_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    cls = RandomForestClassifier(max_depth=4)
    cls.fit(X_train, y_train)
    y_hat = cls.predict(X_test)
    y_prob = cls.predict_proba(X_test)
    return y_test, y_hat

def nn_model(X, y):
    from keras.models import Sequential
    from keras.layers import Dense, Activation, Embedding, LSTM
    import keras 

    X_train, X_test, y_train, y_test = train_test_split(X, y)
        
    model = Sequential()
    n_features = X_train.shape[1]
    model.add(Dense(n_features, input_dim=n_features, activation='relu'))
    model.add(Dense(n_features, activation='relu'))
    model.add(Dense(20, activation='relu'))
    model.add(Dense(15, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(5, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()

    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

   
# results = baseline_model(X, y)

nn_model(X, y)




Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_2 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_3 (Dense)              (None, 20)                500       
_________________________________________________________________
dense_4 (Dense)              (None, 15)                315       
_________________________________________________________________
dense_5 (Dense)              (None, 10)                160       
_________________________________________________________________
dense_6 (Dense)              (None, 5)                 55        
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 6         
Total para

In [163]:


def model_summary(y_test, y_hat):
    from sklearn.metrics import accuracy_score, f1_score
    accuracy = accuracy_score(y_hat, y_test)
    f1 = f1_score(y_hat, y_test)
    
    print("Accuracy is: {}".format(accuracy))
    print("F1 score is: {}".format(f1))
#     plt.scatter(y_prob[:, 1], y_test, alpha=0.01)
    
model_summary(*results)

Accuracy is: 0.5873408239700375
F1 score is: 0.6112483240420576
